# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Clone `CreateDebateScraper` library from github
!git clone https://github.com/utkarsh512/CreateDebateScraper.git
%cd CreateDebateScraper/src/nested/

In [ ]:
# This module is required for core-periphery analysis
!pip install cpnet
!pip install transformers

In [ ]:
from   copy                     import deepcopy
import cpnet
from   itertools                import accumulate
import json
from   matplotlib               import pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import pickle
import re
from   scipy                    import stats
import textwrap
from   thread                   import Comment, Thread
from   tqdm                     import tqdm
nltk.download('punkt') # For tokenizers
nltk.download('stopwords')
import matplotlib
from   nltk.tokenize            import TweetTokenizer
from   nltk.corpus              import stopwords
from   pprint                   import pprint
# import shifterator as sh
# import wordcloud
# import skbio
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams["figure.figsize"] = (12, 5)
STOP_WORDS = list(stopwords.words('english'))

In [ ]:
# Custom routine to clean texts scraped from Web.
# It removes hyperlinks, punctuation marks (except apostrophe)

tknz = TweetTokenizer()

def clean_text(text):
    """
    Preprocessing text
    """
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub("-", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\u2018", "X", text) 
    text = re.sub("\u2019", "X", text) 
    text = re.sub("\'", "X", text) 
    wordTokens_ = tknz.tokenize(text)
    wordTokens = list()
    for x in wordTokens_:
        x = ''.join([v for v in x if v.isalnum() or v == ' '])
        if len(x) > 0 and x != 'X':
            x = x.replace('X', '\'')
            wordTokens.append(x)
    return wordTokens

In [ ]:
comments = dict()

# Topical forums on CreateDebate. We have scraped comments for all of the
# following forurm.
categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

# However, we will be analyzing comments from selected forum only!
# These forum have at least 10k comments each.
categories_selected = ['politics2', 'religion', 'world', 
                       'science', 'law', 'technology']

for x in categories_selected:
    comments[x] = list()

In [ ]:
# Loading comments from select forums

for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')

    # Get all the `Thread` objects pickled while scraping.
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()

    # While classifying CreateDebate comments, we used comments as per author mode.
    # Hence, using the same mode to attach classification score with the comments.
    # 
    # score < 0.5 -> ad hominem comment
    #       > 0.5 -> non ad hominem comment
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append((v, k))
            except:
                authors[v.author] = list()
                authors[v.author].append((v, k))

    ctr = 0
    # Load the classification score of the comments.
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    # Attach classification score with the comments.
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment, cid = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            foo['score'] = cws[ctr][0]
            foo['validation'] = cws[ctr][1][0]
            foo['id'] = int(cid[3:])
            comments[cat].append(foo)
            ctr += 1

In [ ]:
ah_score_comments = dict()

for cat in categories_selected:
    for comment in comments[cat]:
        ah_score_comments[comment['id']] = 1 - comment['score']

In [ ]:
def parse_tstring(tstring):
    """
    Parses comment's time to an integer to enable
    comparison between comments based on their time of posting
    """
    if tstring == 'Not Available':
        raise ValueError('Invalid posting time for parse_tstring')
    tstring = tstring.replace('T', '-').replace(':', '-').replace('+', '-').split('-')
    return int(''.join(tstring[:-2]))

In [ ]:
# Loading AH score

with open('/content/gdrive/MyDrive/Temp/47-ah-score.pkl', 'rb') as fp:
    ah_score = pickle.load(fp)

# `ah_score` is a dictionary that contains the ah score of the comments written
# by all the users

# key: category -> user
# value: list of ah_score for given user for given category

# value > 0.5 --> ad hominem
# value < 0.5 --> non ad hominem

In [ ]:
# Loading CreateDebate profile characteristics into dataframe
df = pd.read_json('/content/gdrive/MyDrive/DL/CreateDebate/profile/results.json', lines=True)

# Extract useful characteristics
reward_points_map = {k : v for k, v in zip(df['username'].tolist(), df['reward_points'].tolist())}
efficiency_map    = {k : v for k, v in zip(df['username'].tolist(), df['efficiency'].tolist())}
allies_map        = {k : len(v) for k, v in zip(df['username'].tolist(), df['allies'].tolist())}
enemies_map       = {k : len(v) for k, v in zip(df['username'].tolist(), df['enemies'].tolist())}
hostiles_map      = {k : len(v) for k, v in zip(df['username'].tolist(), df['hostiles'].tolist())}

In [ ]:
def profile_characteristics_stats(user_subset):
    """
    Returns average and standard deviation of profile characteristics for 
    given subset of users.

    :param user_subset: Iterable containing usernames

    >>> avgs, stds = profile_characterisitics_stat(user_subset)
    >>> rewards_avg, efficiency_avg, n_allies_avg, n_enemies_avg, n_hostiles_avg = avgs
    >>> rewards_std, efficiency_std, n_allies_std, n_enemies_std, n_hostiles_std = stds

    Note that profile characteristics for some users might not be present in our
    dataset as some users might have deleted their account when we scraped the
    forum to obtain these characteristics.
    """
    rewards_ = list()
    efficiency_ = list()
    n_allies = list()
    n_enemies = list()
    n_hostiles = list()

    for user in user_subset:
        try:
            rewards_.append(reward_points_map[user])
        except:pass
        try:
            efficiency_.append(efficiency_map[user])
        except:pass
        try:
            n_allies.append(allies_map[user])
        except:pass
        try:
            n_enemies.append(enemies_map[user])
        except:pass
        try:
            n_hostiles.append(hostiles_map[user])
        except:pass
    
    grpd_data = [rewards_, efficiency_, n_allies, n_enemies, n_hostiles]
    avgs = [np.average(x) for x in grpd_data]
    stds = [np.std(x) for x in grpd_data]
    
    return avgs, stds

In [ ]:
# Maximum ah score per category per author
#   key: category -> author
#   value: maximum ah score

ah_score_max = dict()

for category, author_data in ah_score.items():
    ah_score_max[category] = dict()
    for author, ah_scores in author_data.items():
        ah_score_max[category][author] = np.max(ah_scores)

In [ ]:
comment_count = dict()
# key: category -> author
# value: number of comments written by author in the given forum

for category in categories_selected:
    comment_count[category] = dict()

    for comment in comments[category]:
        author = comment['author']
        try:
            comment_count[category][author] += 1
        except KeyError:
            comment_count[category][author] = 1

In [ ]:
user_list = set()

for category in categories_selected:
    for comment in comments[category]:
        user_list.add(comment['author'])

user_list = list(user_list)

In [ ]:
first_post_time = dict()
# key: category -> user
# value: post time of the first comment by given user in the given category
#        It is an integer as returned by parse_tstring routine

for category in categories_selected:
    first_post_time[category] = dict()

    for comment in comments[category]: 
        if comment['time'] == 'Not Available':
            continue
        author = comment['author']
        try:
            first_post_time[category][author] = min(first_post_time[category][author], parse_tstring(comment['time']))
        except KeyError:
            first_post_time[category][author] = parse_tstring(comment['time'])

In [ ]:
def get_migrated_users(categories_1, categories_2, categories_1_origin=True, require_migration=True):
    """
    Returns a list of usernames who migrated from categories_1 to categories_2

    If categories_1_origin is True, we will consider all other major categories
    to compute post_time_2, so as to ensure that first post by the user is in 
    categories_1

    If require_migration is True, post_time_1 < post_time_2 condition is relaxed
    """

    resultant_list = list()

    for user in user_list:
        post_time_1 = 20220101000000
        post_time_2 = 20220101000000

        if not isinstance(categories_1, set):
            categories_1 = set(categories_1)
        if not isinstance(categories_2, set): 
            categories_2 = set(categories_2)
        
        for category in categories_1:
            try:
                cur_post_time = first_post_time[category][user]
                post_time_1 = min(post_time_1, cur_post_time)
            except KeyError:
                pass
        
        for category in categories_2:
            try:
                cur_post_time = first_post_time[category][user]
                post_time_2 = min(post_time_2, cur_post_time) 
            except KeyError:
                pass

        if post_time_1 == 20220101000000 or post_time_2 == 20220101000000:
            continue

        if categories_1_origin:
            for category in categories_selected:
                if not ((category in categories_1) or (category in categories_2)):
                    try:
                        cur_post_time = first_post_time[category][user]
                        post_time_2 = min(post_time_2, cur_post_time)
                    except KeyError:
                        pass

        if post_time_1 < post_time_2 or not require_migration:
            resultant_list.append(user)
        
    return resultant_list

In [ ]:
def partition_migrated_users(migration_list, categories_1, categories_2):
    """
    Partitions the users into 4 categories: 
        AH-AH
        AH-NonAH
        NonAH-AH
        NonAH-NonAH

    Users are classified as AH in a given category if they post at least one 
    ad hominem comment in that category
    
    Note: migration_list should be obtained using get_migrated_users method
    """

    ah_ah_list = []
    ah_nonah_list = []
    nonah_ah_list = []
    nonah_nonah_list = []

    for user in migration_list:
        max_score_1 = 0
        max_score_2 = 0
        for category in categories_1:
            max_score_1 = max(max_score_1, ah_score_max[category].get(user, 0))
        for category in categories_2:
            max_score_2 = max(max_score_2, ah_score_max[category].get(user, 0))

        if max_score_1 > 0.5 and max_score_2 > 0.5:
            ah_ah_list.append(user)

        elif max_score_1 > 0.5 and max_score_2 < 0.5:
            ah_nonah_list.append(user)
        
        elif max_score_1 < 0.5 and max_score_2 > 0.5:
            nonah_ah_list.append(user)

        elif max_score_1 < 0.5 and max_score_2 < 0.5:
            nonah_nonah_list.append(user)
        
        else:
            print(user)

    return ah_ah_list, ah_nonah_list, nonah_ah_list, nonah_nonah_list 

In [ ]:
# Get a list of all comment thread representative to build support and dispute 
# networks

threads = []

for category in categories_selected:
    reader_addr = f'/content/gdrive/MyDrive/DL/CreateDebate/{category}/threads.log'
    reader = open(reader_addr, 'rb')
    try:
        while True:
            e = pickle.load(reader)
            threads.append(e)
    except:
        reader.close()

In [ ]:
def build_graph(user_subset, n1 = 0, n2 = 0):
    """
    Builds support graph and dispute graph from hyper-parameters n1 and n2
    
    Inputs
    ------
    :param n1: threshold on number of level-1 comments
    :param n2: threshold on number of direct replies

    Output
    ------
    (
        author_map: dict,
        reverse_map: list,
        author_count: int, 
        support_graph: nx.DiGraph,
        support_matrix: list, 
        dispute_graph: nx.DiGraph, 
        dispute_matrix: list
    )
    """

    # Uses globally defined `threads` variable to construct this dictionary.
    # You may choose which categories to be included while building `threads`

    # key  : author name
    # value: count of level-1 comments
    athr = dict()

    for e in threads:
        if 'root' in e.metaL.keys():
            for key in e.metaL['root'].keys():
                cmnt = e.comments[key]
                cur_athr = cmnt.author
                try:
                    athr[cur_athr] += 1
                except:
                    athr[cur_athr] = 1
        if 'root' in e.metaR.keys():
            for key in e.metaR['root'].keys():
                cmnt = e.comments[key]
                cur_athr = cmnt.author
                try:
                    athr[cur_athr] += 1
                except:
                    athr[cur_athr] = 1
    
    # Filter those authors who satisfy the contraint on number of level-1 comments
    L1_athr = dict()
    for x in athr:
        if athr[x] >= n1:
            L1_athr[x] = True

    # Now use `athr` for storing count of direct replies
    # key  : author name
    # value: count of direct replies received
    athr = dict()

    # Depth-first search utility to get number of direct replies for each author
    def dfs(Map, cmntMap, athr, cid='root'):
        if cid == 'root':
            for key in Map[cid].keys():
                dfs(Map[cid], cmntMap, athr, key)
            return

        cur_author = cmntMap[cid].author
        try:
            athr[cur_author] += len(Map[cid].keys())
        except:
            athr[cur_author] = len(Map[cid].keys())

        for key in Map[cid].keys():
            dfs(Map[cid], cmntMap, athr, key)

    # Traverse thread-tree to get number of direct replies for each author
    for e in threads:
        if 'root' in e.metaL.keys():
            dfs(e.metaL, e.comments, athr)
        if 'root' in e.metaR.keys():
            dfs(e.metaR, e.comments, athr) 
    
    # Filter authors who now satify both the contrainsts on count of 
    # - level-1 comments
    # - direct replies
    A = []
    for x in athr:
        if x not in user_subset:
            continue
        if athr[x] >= n2:
            try:
                z = L1_athr[x]
                A.append(x)
            except KeyError:
                pass

    # key  : author name
    # value: corresponing node number in the support/dispute network
    author_map = dict()

    # To get author name for node number
    reverse_map = ["" for _ in range(len(A))]
    author_count = len(A)

    for i in range(author_count):
        author_map[A[i]] = i
        reverse_map[i] = A[i]
    
    # Weighted adjacency matrices for support and dispute network
    # Weight for directed edge b/w Node A and Node B corresponsds to the number
    # of times Node A supported/disputed Node B.
    support_matrix = [[0 for j in range(author_count)] for i in range(author_count)]
    dispute_matrix = [[0 for j in range(author_count)] for i in range(author_count)]

    # Depth-first search utility to build the adjacency matrices for support 
    # and dispute graph. 
    # Here, we use polarity of the comment to identify whether the authors are
    # showing support or dispute.
    def dfs1(Map, cmntMap, cid='root'):
        if cid == 'root':
            for key in Map[cid].keys():
                dfs1(Map[cid], cmntMap, key)
            return

        cur_author = cmntMap[cid].author
        cur_pol = cmntMap[cid].polarity
        
        if cur_author in author_map and cur_pol != 'Not Available':
            cur_author_id = author_map[cur_author]
            for key in Map[cid].keys():
                nxt_author = cmntMap[key].author
                nxt_pol = cmntMap[key].polarity
                if nxt_author in author_map and nxt_pol != 'Not Available':
                    nxt_author_id = author_map[nxt_author]
                    if cur_pol == nxt_pol:
                        support_matrix[nxt_author_id][cur_author_id] += 1
                    else:
                        dispute_matrix[nxt_author_id][cur_author_id] += 1

        for key in Map[cid].keys():
            dfs1(Map[cid], cmntMap, key)

    for e in threads:
        if 'root' in e.metaL:
            dfs1(e.metaL, e.comments)
        if 'root' in e.metaR:
            dfs1(e.metaR, e.comments)
        
    # Create NetworkX graphs from the adjacency matrices.
    # We need nx graphs in order to get various network stats provided in nx
    # library.
    support_graph = nx.DiGraph()
    for i in range(author_count):
        for j in range(author_count):
            if support_matrix[i][j] != 0:
                support_graph.add_weighted_edges_from([(i, j, support_matrix[i][j])])

    dispute_graph = nx.DiGraph()
    for i in range(author_count):
        for j in range(author_count):
            if dispute_matrix[i][j] != 0:
                dispute_graph.add_weighted_edges_from([(i, j, dispute_matrix[i][j])])
    
    return (author_map, reverse_map, author_count, support_graph, support_matrix, dispute_graph, dispute_matrix)

In [ ]:
# Construct global support and dispute network for entire CreateDebate corpus
user_map, user_reverse_map, user_count, support_graph, support_matrix, dispute_graph, dispute_matrix = build_graph(user_list)

In [ ]:
def get_reciprocity_stats(user_subset):
    """
    Returns reciprocity for given subset of users in local support and dispute 
    graphs

    >>> support_r, dispute_r = get_reciprocity_stats(user_subset)
    """
    _, _, _, support_graph_, _, dispute_graph_, _ = build_graph(user_subset)

    try:
        support_graph_r = nx.algorithms.reciprocity(support_graph_)
    except:
        support_graph_r = 0

    try:
        dispute_graph_r = nx.algorithms.reciprocity(dispute_graph_)
    except:
        dispute_graph_r = 0

    return support_graph_r, dispute_graph_r

In [ ]:
# Get dicts containing centrality value for each node from global support and 
# dispute networks. This will be used for computing stats for user subset.
support_centrality_dict = nx.algorithms.centrality.degree_centrality(support_graph)
dispute_centrality_dict = nx.algorithms.centrality.degree_centrality(dispute_graph)

In [ ]:
def get_centrality_stats(user_subset):
    """
    Returns mean and standard deviation of degree centrality for given user 
    subset in global support and dispute networks.

    >>> s_avg, s_std, d_avg, d_std = get_centrality_stats(user_subset)
    """
    s_c = []
    d_c = []

    for user in user_subset:
        try:
            s_c.append(support_centrality_dict[user_map[user]])
        except:
            pass
        try:
            d_c.append(dispute_centrality_dict[user_map[user]])
        except:
            pass
    
    return np.average(s_c), np.std(s_c), np.average(d_c), np.std(d_c)

In [ ]:
# Get dicts containing clustering coeffieient for each node from global support 
# and dispute networks. This will be used for computing stats for user subset.
support_clustering_dict = nx.algorithms.cluster.clustering(support_graph)
dispute_clustering_dict = nx.algorithms.cluster.clustering(dispute_graph)

In [ ]:
def get_clustering_stats(user_subset):
    """
    Returns mean and standard deviation of clustering coefficient for given user 
    subset in global support and dispute networks.

    >>> s_avg, s_std, d_avg, d_std = get_clustering_stats(user_subset)
    """
    s_c = []
    d_c = []

    for user in user_subset:
        try:
            s_c.append(support_clustering_dict[user_map[user]])
        except:
            pass
        try:
            d_c.append(dispute_clustering_dict[user_map[user]])
        except:
            pass
    
    return np.average(s_c), np.std(s_c), np.average(d_c), np.std(d_c)

In [ ]:
def core_periphery(G, normalize=True):
    # Get coreness value for each nodes as dictionary. We use Boyd algorithm.
    # For more info about the algorithm used, see
    # https://github.com/skojaku/core-periphery-detection
    algorithm = cpnet.MINRES()
    algorithm.detect(G)
    coreness = algorithm.get_coreness()
    if normalize:
        max_coreness_value = -1
        min_coreness_value = 1

        for v in coreness.values():
            max_coreness_value = max(max_coreness_value, v)
            min_coreness_value = min(min_coreness_value, v)
        
        diff = max_coreness_value - min_coreness_value
        
        for k in coreness.keys():
            coreness[k] = (coreness[k] - min_coreness_value) / diff

    return coreness        

In [ ]:
# Get dicts containing core-pheriphery values for each node from global support 
# and dispute networks. This will be used for computing stats for user subset.
support_coreness = core_periphery(support_graph)
dispute_coreness = core_periphery(dispute_graph)

In [ ]:
def normalize_dict(x):
    """
    Normalize elements in given dictionary as
        element = (element - min_element) / (max_element - min_element)
    """
    mini = min(x.values())
    maxa = max(x.values())

    res = dict()

    for k, v in x.items():
        res[k] = (v - mini) / (maxa - mini)
    return res

In [ ]:
def normalize_array(x):
    """
    Normalize elements in given array as
        element = (element - min_element) / (max_element - min_element)
    """
    assert isinstance(x, (list, tuple)), "Expected a list or tuple"
    mini = min(x)
    maxa = max(x)
    res = []
    for e in x:
        res.append((e - mini) / (maxa - mini))
    return res

In [ ]:
def get_core_periphery(user_subset, do_local_normalize=False):
    """
    Returns coreness values as lists from given subset of users in
    global support and dispute network.

    >>> s_coreness, d_coreness = get_core_periphery(user_subset)
    """
    s_coreness = []
    d_coreness = []

    for user in user_subset:
        try:
            s_coreness.append(support_coreness[user_map[user]])
        except:
            pass
        try:
            d_coreness.append(dispute_coreness[user_map[user]])
        except:
            pass
        
    if do_local_normalize:
        s_coreness = normalize_array(s_coreness)
        d_coreness = normalize_array(d_coreness)
    
    return s_coreness, d_coreness

In [ ]:
def plot_support_coreness(user_subset, do_local_normalize=False):
    s_coreness, _ = get_core_periphery(user_subset, do_local_normalize)
    plt.hist(s_coreness, bins=[x / 100 for x in range(101)])
    plt.xlabel('Core-ness')
    plt.ylabel('#Users')
    plt.title('Core-ness distribution for support graph')

In [ ]:
def plot_dispute_coreness(user_subset, do_local_normalize=False):
    _, d_coreness= get_core_periphery(user_subset, do_local_normalize)
    plt.hist(d_coreness, bins=[x / 100 for x in range(101)])
    plt.xlabel('Core-ness')
    plt.ylabel('#Users')
    plt.title('Core-ness distribution for dispute graph')

In [ ]:
def get_core_periphery_stats(user_subset, do_local_normalize=False):
    """
    Returns quantile distribution of coreness value for given user subset

    >>> s_dist, d_dist = get_core_periphery_stats(user_subset, do_local_normalize=False)
    """
    s_coreness, d_coreness = get_core_periphery(user_subset, do_local_normalize)

    s_c_q = [0 for _ in range(4)]
    d_c_q = [0 for _ in range(4)]

    for x in s_coreness:
        if x < 0.25:
            s_c_q[0] += 1
        elif x >= 0.25 and x < 0.5:
            s_c_q[1] += 1
        elif x >= 0.5 and x < 0.75:
            s_c_q[2] += 1
        else:
            s_c_q[3] += 1
        
    for x in d_coreness:
        if x < 0.25:
            d_c_q[0] += 1
        elif x >= 0.25 and x < 0.5:
            d_c_q[1] += 1
        elif x >= 0.5 and x < 0.75:
            d_c_q[2] += 1
        else:
            d_c_q[3] += 1
    
    return s_c_q, d_c_q

In [ ]:
def display_stats(user_subset, do_local_normalize=True):
    n                                          = len(user_subset)
    s_reci, d_reci                             = get_reciprocity_stats(user_subset) 
    s_deg_avg, s_deg_std, d_deg_avg, d_deg_std = get_centrality_stats(user_subset)
    s_clu_avg, s_clu_std, d_clu_avg, d_clu_std = get_clustering_stats(user_subset)
    user_chr_avg, user_chr_std                 = profile_characteristics_stats(user_subset) 
    s_quantile_dist, d_quantile_dist           = get_core_periphery_stats(user_subset, 
                                                                          do_local_normalize=do_local_normalize)

    print('Size: %d' % n)
    print('Support graph reciprocity: %.2f' % s_reci)
    print('Dispute graph reciprocity: %.2f' % d_reci)

    print('Support graph degree centrality: %.5f ± %.5f' % (s_deg_avg, s_deg_std))
    print('Dispute graph degree centrality: %.5f ± %.5f' % (d_deg_avg, d_deg_std)) 

    print('Support graph clustering coeff: %.2f ± %.2f' % (s_clu_avg, s_clu_std))
    print('Dispute graph clustering coeff: %.2f ± %.2f' % (d_clu_avg, d_clu_std)) 

    print('Reward points: %.2f ± %.2f' % (user_chr_avg[0], user_chr_std[0]))
    print('Efficiency   : %.2f ± %.2f' % (user_chr_avg[1], user_chr_std[1]))
    print('# Allies     : %.2f ± %.2f' % (user_chr_avg[2], user_chr_std[2]))
    print('# Enemies    : %.2f ± %.2f' % (user_chr_avg[3], user_chr_std[3]))
    print('# Hostiles   : %.2f ± %.2f' % (user_chr_avg[4], user_chr_std[4]))

    print(f'Support coreness quantile: {s_quantile_dist}')
    print(f'Dispute coreness quantile: {d_quantile_dist}')

# Core-periphery

In [ ]:
categories_1 = ['religion']
categories_2 = ['politics2']

In [ ]:
migration_list = get_migrated_users(categories_1, categories_2, categories_1_origin=False, require_migration=False)

In [ ]:
len(migration_list)

In [ ]:
AA, AN, NA, NN = partition_migrated_users(migration_list, categories_1, categories_2)

In [ ]:
print('AH-AH: %d, AH-NONAH: %d, NONAH-AH: %d, NONAH-NONAH: %d' % (len(AA), len(AN), len(NA), len(NN)))

In [ ]:
A = AA + AN
N = NA + NN

In [ ]:
display_stats(AA, True)

In [ ]:
display_stats(AN, True)

In [ ]:
display_stats(NA, True)

In [ ]:
display_stats(NN, True)

In [ ]:
display_stats(A, True)

In [ ]:
display_stats(N, True)

In [ ]:
def get_users_by_quantile_threshold(user_subset, threshold):
    _support_coreness = dict()
    _dispute_coreness = dict()

    for user in user_subset:
        try:
            _support_coreness[user] = support_coreness[user_map[user]]
        except:pass
        try:
            _dispute_coreness[user] = dispute_coreness[user_map[user]]
        except:pass

    _support_coreness = normalize_dict(_support_coreness)
    _dispute_coreness = normalize_dict(_dispute_coreness)

    res = set() 

    for k, v in _support_coreness.items():
        if v > threshold:
            res.add(k)
    for k, v in _dispute_coreness.items(): 
        if v > threshold: 
            res.add(k)
    return res

In [ ]:
A_core_users = get_users_by_quantile_threshold(A, threshold=0.5)
N_core_users = get_users_by_quantile_threshold(N, threshold=0.5)

In [ ]:
display_stats(A_core_users)

In [ ]:
display_stats(N_core_users)

# Polar vs. non-polar posts and comments

In [ ]:
n_polar_posts = dict()
n_nonpolar_posts = dict()
n_polar_comments = dict()
n_nonpolar_comments = dict()

In [ ]:
n_polar_comments_ah = dict()
n_nonpolar_comments_ah = dict()

In [ ]:
AA_polar_comments = dict()
AA_nonpolar_comments = dict()

AN_polar_comments = dict()
AN_nonpolar_comments = dict()

NA_polar_comments = dict()
NA_nonpolar_comments = dict()

NN_polar_comments = dict()
NN_nonpolar_comments = dict()

In [ ]:
AA = set(AA)
AN = set(AN)
NA = set(NA)
NN = set(NN)

In [ ]:
for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')

    n_polar_posts[cat] = 0
    n_nonpolar_posts[cat] = 0 
    n_polar_comments[cat] = 0 
    n_nonpolar_comments[cat] = 0 

    n_polar_comments_ah[cat] = 0
    n_nonpolar_comments_ah[cat] = 0

    # Get all the `Thread` objects pickled while scraping.
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()

    for thread in threads:
        if len(thread.metaR) == 0:
            n_nonpolar_posts[cat] += 1
            n_nonpolar_comments[cat] += len(thread.comments)

            for cid in thread.comments.keys():
                try:
                    if ah_score_comments[int(cid[3:])] > 0.5:
                        n_nonpolar_comments_ah[cat] += 1
                except:pass
            
            for comment in thread.comments.values():
                athr = comment.author
                if athr in AA:
                    try:
                        AA_nonpolar_comments[athr] += 1
                    except:
                        AA_nonpolar_comments[athr] = 1
                elif athr in AN:
                    try:
                        AN_nonpolar_comments[athr] += 1
                    except:
                        AN_nonpolar_comments[athr] = 1
                elif athr in NA:
                    try:
                        NA_nonpolar_comments[athr] += 1
                    except:
                        NA_nonpolar_comments[athr] = 1
                elif athr in NN:
                    try:
                        NN_nonpolar_comments[athr] += 1
                    except:
                        NN_nonpolar_comments[athr] = 1

        else:
            n_polar_posts[cat] += 1
            n_polar_comments[cat] += len(thread.comments)

            for cid in thread.comments.keys():
                try:
                    if ah_score_comments[int(cid[3:])] > 0.5:
                        n_polar_comments_ah[cat] += 1
                except:pass
            
            for comment in thread.comments.values():
                athr = comment.author
                if athr in AA:
                    try:
                        AA_polar_comments[athr] += 1
                    except:
                        AA_polar_comments[athr] = 1
                elif athr in AN:
                    try:
                        AN_polar_comments[athr] += 1
                    except:
                        AN_polar_comments[athr] = 1
                elif athr in NA:
                    try:
                        NA_polar_comments[athr] += 1
                    except:
                        NA_polar_comments[athr] = 1
                elif athr in NN:
                    try:
                        NN_polar_comments[athr] += 1
                    except:
                        NN_polar_comments[athr] = 1

In [ ]:
for k, v in n_polar_posts.items():
    print(f'{k} - {v}')

In [ ]:
for k, v in n_nonpolar_posts.items():
    print(f'{k} - {v}')

In [ ]:
for k, v in n_polar_comments.items():
    print(f'{k} - {v}')

In [ ]:
for k, v in n_nonpolar_comments.items():
    print(f'{k} - {v}')

In [ ]:
for k, v in n_polar_comments_ah.items():
    print(f'{k} - {v}')

In [ ]:
for k, v in n_nonpolar_comments_ah.items():
    print(f'{k} - {v}')

In [ ]:
def get_polarity_stats(group):
    pol  = 'list(' + group + '_polar_comments.values())'
    npol = 'list(' + group + '_nonpolar_comments.values())'

    p_avg = np.mean(eval(pol))
    p_std = np.std(eval(pol))

    np_avg = np.mean(eval(npol))
    np_std = np.std(eval(npol))

    print(f'Group: {group}')
    print(f'# Polar comments: {p_avg: .2f} ± {p_std: .2f}')
    print(f'# Non-polar comments: {np_avg: .2f} ± {np_std: .2f}')

In [ ]:
get_polarity_stats('AA')

In [ ]:
get_polarity_stats('AN')

In [ ]:
get_polarity_stats('NA')

In [ ]:
get_polarity_stats('NN')